# Non-Spark Cleaning + Modeling (Benchmark Notebook)

This notebook is intentionally streamlined (no EDA visuals) for fair workflow benchmarking.

## Included benchmarking controls
- Fixed split and random seed (`test_size=0.2`, `random_state=42`)
- Same evaluation metrics for all models (R2, MAE, RMSE)
- End-to-end runtime timing
- Lightweight CPU/memory snapshots before and after major stages

In [1]:
import os
import time
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

try:
    import psutil
except ImportError:
    psutil = None


def resource_snapshot(tag: str):
    if psutil is None:
        print(f"[{tag}] psutil not installed (pip install psutil for CPU/memory snapshots)")
        return
    cpu_pct = psutil.cpu_percent(interval=0.3)
    mem_pct = psutil.virtual_memory().percent
    print(f"[{tag}] CPU%: {cpu_pct:.1f} | Memory%: {mem_pct:.1f}")

In [2]:
RANDOM_STATE = 42
TEST_SIZE = 0.2

employee_path = "../Downloads/Employee_dataset.csv"
salary_path = "../Downloads/Employee_salaries.csv"

run_start = time.perf_counter()
resource_snapshot("start")

[start] CPU%: 11.2 | Memory%: 30.8


In [3]:
# Update these paths if needed
employee_path = r"C:\Users\Alex\Downloads\Employee_dataset.csv"
salary_path = r"C:\Users\Alex\Downloads\Employee_salaries.csv"

assert os.path.exists(employee_path), f"Missing file: {employee_path}"
assert os.path.exists(salary_path), f"Missing file: {salary_path}"

emp = pd.read_csv(employee_path, dtype="string")
sal = pd.read_csv(salary_path, dtype="string")

# Standardize columns
emp = emp.rename(columns={
    "jobId": "job_id",
    "companyId": "company_id",
    "jobRole": "job_role",
    "Industry": "industry",
    "yearsExperience": "years_experience",
    "distanceFromCBD": "distance_from_cbd",
})
sal = sal.rename(columns={"jobId": "job_id", "salaryInThousands": "salary_in_thousands"})

print("Loaded rows:", len(emp), len(sal))

Loaded rows: 1000000 1000000


In [4]:
# ---- Cleaning (mirrors non-spark EDA logic) ----
na_tokens = ["", " ", "NA", "NaN", "nan", "na", "None"]

emp["job_id"] = pd.to_numeric(
    emp["job_id"].astype("string").str.replace("JOB", "", regex=False).replace(na_tokens, pd.NA),
    errors="coerce",
).astype("Int64")

emp["company_id"] = pd.to_numeric(
    emp["company_id"].astype("string").str.replace("COMP", "", regex=False).replace(na_tokens, pd.NA),
    errors="coerce",
).astype("Int64")

for c in ["education", "major"]:
    emp[c] = emp[c].replace(na_tokens, pd.NA).fillna("NONE")

for c in ["job_role", "industry"]:
    emp[c] = emp[c].astype("string").str.strip().str.upper().replace(na_tokens, pd.NA)

emp["years_experience"] = pd.to_numeric(emp["years_experience"], errors="coerce")
emp["distance_from_cbd"] = pd.to_numeric(emp["distance_from_cbd"], errors="coerce")

# Keep complete critical fields
emp = emp.dropna(subset=["job_id", "company_id", "job_role", "industry", "years_experience", "distance_from_cbd"])

# Remove noisy/invalid role entries found during EDA
emp = emp[emp["job_role"] != "SCAMMER"]

# Basic range filters
emp = emp[(emp["years_experience"] >= 0) & (emp["years_experience"] <= 50)]
emp = emp[(emp["distance_from_cbd"] >= 0) & (emp["distance_from_cbd"] <= 100)]

# Salary cleaning
sal["job_id"] = pd.to_numeric(
    sal["job_id"].astype("string").str.replace("JOB", "", regex=False).replace(na_tokens, pd.NA),
    errors="coerce",
).astype("Int64")
sal["salary_in_thousands"] = pd.to_numeric(sal["salary_in_thousands"], errors="coerce")
sal = sal.dropna(subset=["job_id", "salary_in_thousands"])
sal = sal[(sal["salary_in_thousands"] > 0) & (sal["salary_in_thousands"] <= 500)]

# Cast
emp["job_id"] = emp["job_id"].astype("int64")
emp["company_id"] = emp["company_id"].astype("int64")
emp["years_experience"] = emp["years_experience"].astype("int64")
emp["distance_from_cbd"] = emp["distance_from_cbd"].astype("int64")
sal["job_id"] = sal["job_id"].astype("int64")
sal["salary_in_thousands"] = sal["salary_in_thousands"].astype("int64")

# Merge + de-dup
df = emp.merge(sal, on="job_id", how="inner").drop_duplicates().drop_duplicates(subset=["job_id"]).reset_index(drop=True)

print("Cleaned + merged rows:", len(df))
resource_snapshot("post_clean")

Cleaned + merged rows: 999465
[post_clean] CPU%: 8.6 | Memory%: 31.5


In [5]:
# ---- Feature Engineering ----
edu_map = {"NONE": 0, "HIGH_SCHOOL": 1, "BACHELORS": 2, "MASTERS": 3, "DOCTORAL": 4}
role_map = {"JANITOR": 0, "JUNIOR": 1, "SENIOR": 2, "MANAGER": 3, "VICE_PRESIDENT": 4, "CTO": 5, "CFO": 5, "CEO": 6}
industry_map = {"EDUCATION": 1, "SERVICE": 1, "AUTO": 2, "HEALTH": 3, "WEB": 4, "FINANCE": 5, "OIL": 5}
major_map = {"NONE": 0, "LITERATURE": 1, "BIOLOGY": 2, "CHEMISTRY": 3, "PHYSICS": 4, "COMPSCI": 5, "MATH": 6, "BUSINESS": 7, "ENGINEERING": 8}

df["education_level"] = df["education"].map(edu_map)
df["job_role_rank"] = df["job_role"].map(role_map)
df["industry_score"] = df["industry"].map(industry_map)
df["major_score"] = df["major"].map(major_map)
df["handcrafted_score"] = df[["education_level", "job_role_rank", "industry_score", "major_score"]].sum(axis=1)
df["exp_sq"] = df["years_experience"] ** 2

feature_cols = [
    "years_experience",
    "distance_from_cbd",
    "education_level",
    "job_role_rank",
    "industry_score",
    "major_score",
    "handcrafted_score",
    "exp_sq",
]
target_col = "salary_in_thousands"

model_df = df.dropna(subset=feature_cols + [target_col]).copy()
print("Model rows:", len(model_df))

Model rows: 999464


In [6]:
# ---- Split + Train + Evaluate (consistent metrics) ----
X = model_df[feature_cols]
y = model_df[target_col]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE
)


def eval_predictions(y_true, y_pred):
    return {
        "R2": r2_score(y_true, y_pred),
        "MAE": mean_absolute_error(y_true, y_pred),
        "RMSE": np.sqrt(mean_squared_error(y_true, y_pred)),
    }

rows = []

# Linear Regression
t0 = time.perf_counter()
lr = LinearRegression()
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_test)
lr_time = time.perf_counter() - t0
rows.append({"workflow": "non_spark", "model": "LinearRegression", "runtime_seconds": lr_time, **eval_predictions(y_test, lr_pred)})

# Random Forest
t0 = time.perf_counter()
rf = RandomForestRegressor(n_estimators=120, max_depth=14, random_state=RANDOM_STATE, n_jobs=-1)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)
rf_time = time.perf_counter() - t0
rows.append({"workflow": "non_spark", "model": "RandomForest", "runtime_seconds": rf_time, **eval_predictions(y_test, rf_pred)})

results = pd.DataFrame(rows).sort_values("RMSE").reset_index(drop=True)
results

,workflow,model,runtime_seconds,R2,MAE,RMSE
0,non_spark,RandomForest,15.963716,0.752838,15.575233,19.242555
1,non_spark,LinearRegression,0.078382,0.741160,15.928455,19.691869


In [7]:
# Prediction sample (for sanity check)
best_name = results.iloc[0]["model"]
best_pred = rf_pred if best_name == "RandomForest" else lr_pred

pred_sample = pd.DataFrame({
    "actual_salary_k": y_test.values[:15],
    "predicted_salary_k": best_pred[:15]
})

print("Best model:", best_name)
pred_sample

Best model: RandomForest


,actual_salary_k,predicted_salary_k
0,154,162.692019
1,87,118.946030
2,177,141.940136
3,121,146.435814
4,136,174.069779
5,99,85.189909
6,119,132.097704
7,111,129.307963
8,111,97.628029
9,110,144.015392


In [8]:
run_total = time.perf_counter() - run_start
resource_snapshot("end")
print(f"Total notebook runtime: {run_total:.2f} seconds")

[end] CPU%: 4.7 | Memory%: 32.2
Total notebook runtime: 23.49 seconds
